In [1]:
import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
trainData = datasets.MNIST(root='data', train= True, download= True, transform= ToTensor())

In [3]:
testData = datasets.MNIST(root='data', train= False, download= True, transform= ToTensor())

In [4]:
print(f"{testData} \n\n {trainData}")

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor() 

 Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [5]:
trainLoader = torch.utils.data.DataLoader(trainData, batch_size= 64, shuffle= True)
testLoader = torch.utils.data.DataLoader(testData, batch_size= 100, shuffle=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convLayer1 = nn.Conv2d(in_channels= 1, out_channels= 10, kernel_size=5)
        self.convLayer2 = nn.Conv2d(in_channels= 10, out_channels= 20,kernel_size=5)
        self.conv2Drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.convLayer1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2Drop(self.convLayer2(x)), 2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x)

In [7]:

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Currently used device: {device}")

Currently used device: mps


In [8]:
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
calcLoss = nn.CrossEntropyLoss()

In [21]:
def train(epoch):
    model.train()
    for batchIndex, (data, target) in enumerate(trainLoader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = calcLoss(output, target)
        loss.backward()
        optimizer.step()
        # TODO Print for epoch

In [29]:
def test():
    model.eval()
    testLoss = 0
    correctVal = 0

    with torch.no_grad():
        for data, target in testLoader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            testLoss += F.nll_loss(input= output, target= target, size_average=False).item()
            prediction = output.data.max(1, keepdim=True)[1]
            correctVal += prediction.eq(target.data.view_as(prediction)).sum()

    testLoss /= len(testLoader.dataset)
    print(f"Average loss: {testLoss:.4f}, accuracy {correctVal}/{len(testLoader.dataset)} ({100. * correctVal / len(testLoader.dataset):.0f}%)\n")
            

In [30]:
for epoch in range(1, 4):
    train(epoch)
    test()

/var/folders/xs/m589rd993f3_8k0zh1qk8vkw0000gn/T/ipykernel_90201/2227137575.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/Users/jendras/Prywatne/Digit Recognizer/.venv/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Average loss: 0.1875, accuracy 9449/10000 (94%
)
Average loss: 0.1660, accuracy 9503/10000 (95%
)
Average loss: 0.1522, accuracy 9534/10000 (95%
)
